<a href="https://colab.research.google.com/github/kri8tinam/Low-Resource-Deontic-Modality-Classification-in-EU-Legislation/blob/main/FUSION.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Libraries

In [1]:
#loading and visualizing
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import re
import string
import torch
import os
import random as rn

#preprocessing
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split

#tokenization
import nltk
from nltk.tokenize import word_tokenize
from nltk.tokenize import sent_tokenize


!pip install transformers
from transformers import AutoTokenizer, AutoModel

try:
  %tensorflow_version 2.x
except Exception:
  pass

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras import backend as K
from tensorflow.keras import callbacks

!pip install keras_lr_finder
!pip install torch_optimizer

import torch.optim as optim
import torch.optim.lr_scheduler as lr_scheduler

from keras_lr_finder import LRFinder
# from clr_callback import CyclicLR


!pip install tensorflow-model-optimization
import tensorflow_model_optimization as tfmot

!pip install keras-tuner
import keras_tuner
from kerastuner.tuners import RandomSearch
from kerastuner.engine.hyperparameters import HyperParameters

from tensorflow.keras import regularizers
import time

# evaluation
from sklearn.metrics import accuracy_score, precision_score, f1_score, confusion_matrix

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 25.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 35.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 60.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 65.6 MB/s eta 0:00:00
Colab only includes TensorFlow 2.x; %tensorflow_version has no effect.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.9/61.9 kB 2.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 241.2/241.2 kB 5.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 176.1/176.1 kB 3.4 MB/s eta 0:00:00


<ipython-input-1-0d7e7f60ed9c>:53: DeprecationWarning: `import kerastuner` is deprecated, please use `import keras_tuner`.
  from kerastuner.tuners import RandomSearch


In [3]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


# Data Loading

In [4]:
train_data = pd.read_csv('/content/drive/MyDrive/TRAIN.csv', delimiter= ',')
test_data = pd.read_csv('/content/drive/MyDrive/TEST.csv', delimiter= ',')

# Samples
train100 = pd.read_csv('/content/drive/MyDrive/train100.csv', delimiter= ',')
train200 = pd.read_csv('/content/drive/MyDrive/train200.csv', delimiter= ',')
train300 = pd.read_csv('/content/drive/MyDrive/train300.csv', delimiter= ',')
train350 = pd.read_csv('/content/drive/MyDrive/train350.csv', delimiter= ',')
train400 = pd.read_csv('/content/drive/MyDrive/train400.csv', delimiter= ',')
train450 = pd.read_csv('/content/drive/MyDrive/train450.csv', delimiter= ',')

# Filtered & Augmented
filtered_150 = pd.read_csv('/content/drive/MyDrive/train_filtered_150.csv', delimiter= ',')
filtered_312 = pd.read_csv('/content/drive/MyDrive/train_filtered_312.csv', delimiter= ',')
aug_train150 = pd.read_csv('/content/drive/MyDrive/aug_train150.csv', delimiter= ',')
aug_train312 = pd.read_csv('/content/drive/MyDrive/aug_train312.csv', delimiter= ',')

In [5]:
test_data

,text,category
0,Organizations must ensure that their product r...,obligations
1,For a period of thirty years from 1 October 20...,right
2,Member States may adopt measures to limit acce...,power
3,For a period of seventeen years from 1 October...,right
4,Organizations must ensure that their product r...,obligations
...,...,...
321,Member States may adopt measures to limit acce...,power
322,Organizations must ensure that their product a...,obligations
323,The Union shall ensure the right to protection...,prohibitions
324,For a period of twenty years from 1 November 2...,power


In [6]:
print(train_data['category'].value_counts())
print(test_data['category'].value_counts())

permission     117
prohibition    108
obligation      85
omission        84
power           74
right           47
Name: category, dtype: int64
obligations     71
permission      66
right           60
prohibitions    58
power           54
omission        17
Name: category, dtype: int64


# Data Preparation

In [ ]:
char_list = [r'\d', '\+', ':', '\\n', '#', '$', '%', '&','"', ',', '-',
             ';', '<', '=', '>', '@', '!', '^', '_', '`', '{', '|', '}', '~']   # list of special characters/punctuation

def char_search(text, col, characters):
  for char in characters:
    count = text[col].str.contains(char).sum()
    print(f'{char} special character is present in {count} entries')

char_search(test_data, 'text', char_list)

In [ ]:
def text_cleanse(text, col, characters):
  text[col] = text[col].str.lower() # convert to lowercase
  for char in characters:
    text[col] = text[col].str.replace(char, '', regex = True) # remove special characters

text_cleanse(test_data, 'text', char_list)

char_search(test_data, 'text', char_list)

In [9]:
# CLEANING THE DATASETS

def cleaning(text):
  text = text.replace('$', '')
  text = text.replace('^', '')
  text = re.sub(r'\n\n', '. ', text)
  text = re.sub(r"\(\d+\)", "", text)
  text = text.replace('   ', " ")
  text = re.sub(r"\b\d\. \b", "", text)
  text = text.strip()

  return text

 Label Encoding

In [10]:
le = LabelEncoder()
train_data['category'] = le.fit_transform(train_data["category"])
test_data['category'] = le.fit_transform(test_data["category"])

filtered_150['category'] = le.fit_transform(filtered_150["category"])
filtered_312['category'] = le.fit_transform(filtered_312["category"])
aug_train150['category'] = le.fit_transform(aug_train150["category"])
aug_train312['category'] = le.fit_transform(aug_train312["category"])

train100['category'] = le.fit_transform(train100["category"])
train200['category'] = le.fit_transform(train200["category"])
train300['category'] = le.fit_transform(train300["category"])
train350['category'] = le.fit_transform(train350["category"])
train400['category'] = le.fit_transform(train400["category"])
train450['category'] = le.fit_transform(train450["category"])

# Fusion model

## Benchmark

CNN + A + BILSTM + CNN

*   optimizer - RMSProp
*   batch size = 50


In [11]:
def reproduceResult():
  seed_value= 0


  with tf.device("/cpu:0"):
    ...


  os.environ['PYTHONHASHSEED']=str(seed_value)
  np.random.seed(0)
  rn.seed(0)


  session_conf = tf.compat.v1.ConfigProto(intra_op_parallelism_threads=1,
                                          inter_op_parallelism_threads=1)


  tf.compat.v1.set_random_seed(seed_value)
  sess = tf.compat.v1.Session(graph=tf.compat.v1.get_default_graph(), config=session_conf)
  tf.compat.v1.keras.backend.set_session(sess)
  tf.compat.v1.keras.backend.clear_session()

reproduceResult()
import tempfile
import zipfile


In [ ]:
reproduceResult()


import nltk
nltk.download('stopwords')
nltk.download('wordnet')

import re
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords


stop_words = set(stopwords.words("english"))
lemmatizer = WordNetLemmatizer()


def lemmatization(text):
    text = text.lower()
    text = [lemmatizer.lemmatize(token) for token in text.split(" ")]
    text = " ".join(text)
    return text

train_data['lemmatized text'] = train_data.text.apply(lambda x: lemmatization(x))
test_data['lemmatized text'] = test_data.text.apply(lambda x: lemmatization(x))



In [ ]:
# MAX_FEATURES = 6000
embed_num_dims = 300
tokenizer = Tokenizer()
tokenizer.fit_on_texts(train_data['lemmatized text'])
index_of_words = tokenizer.word_index
vocab_size = len(index_of_words) + 1

tokenized_train = tokenizer.texts_to_sequences(train_data['lemmatized text'])
tokenized_test = tokenizer.texts_to_sequences(test_data['lemmatized text'])


RNN_CELL_SIZE = 32

MAX_LEN = 130
X_train1 = pad_sequences(tokenized_train, maxlen=MAX_LEN,padding='pre')
y_train1 = train_data['category']

X_test1 = pad_sequences(tokenized_test, maxlen=MAX_LEN,padding='pre')
y_test1 = test_data['category']

print(X_train1.shape)
print(X_test1.shape)
print(y_train1.shape)
print(y_test1.shape)


In [14]:
def create_embedding_matrix(word_index, embedding_dim):
    vocab_size = len(word_index) + 1  # Adding again 1 because of reserved 0 index
    embedding_matrix = np.zeros((vocab_size, embedding_dim))
    with open('/content/drive/MyDrive/cc.en.300.vec') as f:
        for line in f:
            word, *vector = line.split()
            if word in word_index:
                idx = word_index[word]
                embedding_matrix[idx] = np.array(
                    vector, dtype=np.float32)[:embedding_dim]
    return embedding_matrix


In [ ]:
embedd_matrix = create_embedding_matrix(index_of_words, embed_num_dims)

LOG_DIR = f"{int(time.time())}"
seed_value= 0

def benchmark(hp):

  reproduceResult()

  print('Ya it comes here')

  # SEARCH VALUES
  # Adjust these values later based on the reported 'best so far' values
  att_size = hp.Int("attention size",min_value =16, max_value = 128, step = 16)
  cnn_1_size = hp.Int("cnn1 size",min_value =16, max_value = 96, step = 16)
  cnn_1_dropout = hp.Float("cnn1 dropout",min_value = 0.1,max_value = 0.3,step = 0.1)

  lstm_size = hp.Int("lstm size",min_value =32, max_value = 256, step = 32)
  lstm_dropout = hp.Float("lstm dropout",min_value = 0.1,max_value = 0.5,step = 0.1)
  cnn_2_size = hp.Int("cnn2 size",min_value =32, max_value = 256, step = 32)
  cnn_2_dropout = hp.Float("cnn2 dropout",min_value = 0.1,max_value = 0.5,step = 0.1)

  lr = hp.Float("learning rate",min_value = 2e-5 ,max_value = 1e-3,step = 2e-5)



  # INPUT DEFINITION
  seq_input = keras.layers.Input(shape=(MAX_LEN,))

  # EMBEDDING
  embedded = keras.layers.Embedding(vocab_size,
                          embed_num_dims,
                          input_length = MAX_LEN,
                          weights = [embedd_matrix])(seq_input)

  # 1ST CNN LAYER
  cnn_1 = keras.layers.Conv1D(cnn_1_size,3)(embedded)
  cnn_1 = keras.layers.Activation(activation='relu')(cnn_1)
  cnn_1 = keras.layers.BatchNormalization()(cnn_1)
  cnn_1 = keras.layers.Dropout(cnn_1_dropout,seed=seed_value)(cnn_1)

  # ATTENTION
  attention = keras.layers.TimeDistributed(keras.layers.Dense(att_size))(cnn_1)
  attention = keras.layers.Reshape((128,att_size))(attention)
  attention = keras.layers.Activation('relu', name = 'cnn1_attention')(attention)
  attention_output = keras.layers.Dot(axes = 1)([cnn_1, attention])

  # BILSTM LAYER
  lstm = keras.layers.Bidirectional(keras.layers.LSTM(lstm_size,
                                                      return_sequences=True,input_shape =(128,)))(attention_output)
  lstm = keras.layers.Activation(activation='relu')(lstm)
  lstm = keras.layers.BatchNormalization()(lstm)
  lstm = keras.layers.Dropout(lstm_dropout,seed=seed_value)(lstm)

  # 2ND CNN LAYER
  cnn_2 = keras.layers.Conv1D(cnn_2_size,3)(lstm)
  cnn_2 = keras.layers.Activation(activation='relu')(cnn_2)
  cnn_2 = keras.layers.BatchNormalization()(cnn_2)
  cnn_2 = keras.layers.Dropout(cnn_2_dropout,seed=seed_value)(cnn_2)

  max_pooling = keras.layers.GlobalMaxPooling1D()(cnn_2)
  output = keras.layers.Dense(6, activation='softmax')(max_pooling)

  model = keras.Model(inputs = [seq_input], outputs = output)
  model.compile(loss='sparse_categorical_crossentropy', optimizer = keras.optimizers.RMSprop(learning_rate = lr), metrics=['accuracy'])

  return model


# EARLY STOPPING
stop_early = tf.keras.callbacks.EarlyStopping(monitor='val_accuracy',
                              patience=4,
                              restore_best_weights=True,
                              verbose=0, mode='max')

# PARAMETER SEARCH
tuner = RandomSearch(
    benchmark,
    objective = keras_tuner.Objective('val_accuracy', direction="max"),
    max_trials = 10,
    executions_per_trial = 1,
    directory = LOG_DIR
    )

tuner.search(x=X_train1,y = y_train1,epochs = 20, batch_size = 50, validation_data = (X_test1,y_test1),callbacks = [stop_early])


tuner.results_summary()

## Minimal Sample

In [16]:
train100['lemmatized text'] = train100.text.apply(lambda x: lemmatization(x))
train200['lemmatized text'] = train200.text.apply(lambda x: lemmatization(x))
train300['lemmatized text'] = train300.text.apply(lambda x: lemmatization(x))
train350['lemmatized text'] = train350.text.apply(lambda x: lemmatization(x))
train400['lemmatized text'] = train400.text.apply(lambda x: lemmatization(x))
train450['lemmatized text'] = train450.text.apply(lambda x: lemmatization(x))

### Original train data

In [ ]:
embed_num_dims = 300
tokenizer = Tokenizer()
tokenizer.fit_on_texts(train_data['lemmatized text'])
index_of_words = tokenizer.word_index
vocab_size = len(index_of_words) + 1

tokenized_train = tokenizer.texts_to_sequences(train_data['lemmatized text'])
tokenized_test = tokenizer.texts_to_sequences(test_data['lemmatized text'])


RNN_CELL_SIZE = 32

MAX_LEN = 130
X_train1 = pad_sequences(tokenized_train, maxlen=MAX_LEN,padding='pre')
y_train1 = train_data['category']

X_test1 = pad_sequences(tokenized_test, maxlen=MAX_LEN,padding='pre')
y_test1 = test_data['category']


print(X_train1.shape)
print(X_test1.shape)
print(y_train1.shape)
print(y_test1.shape)


def create_embedding_matrix(word_index, embedding_dim):
    vocab_size = len(word_index) + 1  # Adding again 1 because of reserved 0 index
    embedding_matrix = np.zeros((vocab_size, embedding_dim))
    with open('/content/drive/MyDrive/cc.en.300.vec') as f:
        for line in f:
            word, *vector = line.split()
            if word in word_index:
                idx = word_index[word]
                embedding_matrix[idx] = np.array(
                    vector, dtype=np.float32)[:embedding_dim]
    return embedding_matrix


embedd_matrix = create_embedding_matrix(index_of_words, embed_num_dims)

def build_model(hp):

  reproduceResult()

  print('Ya it comes here')

  # SEARCH VALUES
  unit_attention = hp.Int("attention_unit",min_value =16, max_value = 128, step = 16)
  cnn_1_unit = hp.Int("cnn_1_unit",min_value =16, max_value = 96, step = 16)
  cnn_1_dropout = hp.Float("cnn_1_dropout",min_value = 0.1,max_value = 0.3,step = 0.1)

  lstm_unit = hp.Int("lstm_unit",min_value =32, max_value = 256, step = 32)
  lstm_dropout = hp.Float("lstm_dropout",min_value = 0.1,max_value = 0.5,step = 0.1)
  cnn_2_unit = hp.Int("cnn_2_unit",min_value =32, max_value = 256, step = 32)
  cnn_2_dropout = hp.Float("cnn_2_dropout",min_value = 0.1,max_value = 0.5,step = 0.1)

  lr = hp.Float("learning rate",min_value = 1e-5 ,max_value = 1e-3,step = 2e-5)


  # INPUT DEFINITION
  seq_input = keras.layers.Input(shape=(MAX_LEN,))

  # EMBEDDING
  embedded = keras.layers.Embedding(vocab_size,
                          embed_num_dims,
                          input_length = MAX_LEN,
                          weights = [embedd_matrix])(seq_input)


  # 1ST CNN LAYER
  cnn = keras.layers.Conv1D(cnn_1_unit,3)(embedded)
  cnn = keras.layers.Activation(activation='relu')(cnn)
  cnn = keras.layers.BatchNormalization()(cnn)
  cnn = keras.layers.Dropout(cnn_1_dropout,seed=seed_value)(cnn)

  # ATTENTION
  attention_vec = keras.layers.TimeDistributed(keras.layers.Dense(unit_attention))(cnn)
  attention_vec = keras.layers.Reshape((128,unit_attention))(attention_vec)
  attention_vec = keras.layers.Activation('relu', name = 'cnn_attention_vec')(attention_vec)
  attention_output = keras.layers.Dot(axes = 1)([cnn, attention_vec])

  # BILSTM LAYER
  lstm = keras.layers.Bidirectional(keras.layers.LSTM(lstm_unit,
                                                      return_sequences=True,input_shape =(128,)))(attention_output)
  lstm = keras.layers.Activation(activation='relu')(lstm)
  lstm = keras.layers.BatchNormalization()(lstm)
  lstm = keras.layers.Dropout(lstm_dropout,seed=seed_value)(lstm)

  # 2ND CNN LAYER
  cnn_2 = keras.layers.Conv1D(cnn_2_unit,3)(lstm)
  cnn_2 = keras.layers.Activation(activation='relu')(cnn_2)
  cnn_2 = keras.layers.BatchNormalization()(cnn_2)
  cnn_2 = keras.layers.Dropout(cnn_2_dropout,seed=seed_value)(cnn_2)

  max_pooling = keras.layers.GlobalMaxPooling1D()(cnn_2)
  output = keras.layers.Dense(6, activation='softmax')(max_pooling)

  model = keras.Model(inputs = [seq_input], outputs = output)
  model.compile(loss = 'sparse_categorical_crossentropy', optimizer = keras.optimizers.RMSprop(learning_rate = lr), metrics = ['accuracy'])

  return model



# Different batch sizes to test
batch_sizes = (4,8,16,32,50)

# Dictionary to store validation accuracy and hyperparameters for each batch size
results = {}

for batch in batch_sizes:
  print('Batch size =', batch)

  LOG_DIR = f"{int(time.time())}"
  seed_value= 0
  stop_early = tf.keras.callbacks.EarlyStopping(monitor='val_accuracy',
                              patience=4,
                              restore_best_weights=True,
                              verbose=0, mode='max')
  tuner = RandomSearch(
    build_model,
    objective = keras_tuner.Objective('val_accuracy', direction="max"),
    max_trials = 10,
    executions_per_trial = 1,
    directory = LOG_DIR
    )

  tuner.search(x=X_train1,y = y_train1,epochs = 20, batch_size = batch, validation_data = (X_test1,y_test1),callbacks = [stop_early])

  # Get the best hyperparameters and validation accuracy for the current batch size
  best_hyperparameters = tuner.get_best_hyperparameters()[0]
  best_val_accuracy = tuner.oracle.get_best_trials(1)[0].score

  # Store the results in the dictionary
  results[batch] = {'best_val_accuracy': best_val_accuracy, 'best_hyperparameters': best_hyperparameters}


print('The results for the original train data using different batch sizes:')
results


### 100 samples

In [ ]:
# MAX_FEATURES = 6000

embed_num_dims = 300
tokenizer = Tokenizer()
tokenizer.fit_on_texts(train100['lemmatized text'])

index_of_words = tokenizer.word_index
vocab_size = len(index_of_words) + 1

tokenized_train = tokenizer.texts_to_sequences(train100['lemmatized text'])
tokenized_test = tokenizer.texts_to_sequences(test_data['lemmatized text'])


RNN_CELL_SIZE = 32

MAX_LEN = 130
X_train1 = pad_sequences(tokenized_train, maxlen=MAX_LEN,padding='pre')
y_train1 = train100['category']

X_test1 = pad_sequences(tokenized_test, maxlen=MAX_LEN,padding='pre')
y_test1 = test_data['category']



print(X_train1.shape)
print(X_test1.shape)
print(y_train1.shape)
print(y_test1.shape)

embedd_matrix = create_embedding_matrix(index_of_words, embed_num_dims)

# Different batch sizes to test
batch_sizes = (4,8,16,32,50)

# Dictionary to store validation accuracy and hyperparameters for each batch size
results100 = {}

for batch in batch_sizes:
  print('Batch size =', batch)

  LOG_DIR = f"{int(time.time())}"
  seed_value= 0
  stop_early = tf.keras.callbacks.EarlyStopping(monitor='val_accuracy',
                              patience=4,
                              restore_best_weights=True,
                              verbose=0, mode='max')
  tuner = RandomSearch(
    build_model,
    objective = keras_tuner.Objective('val_accuracy', direction="max"),
    max_trials = 10,
    executions_per_trial = 1,
    directory = LOG_DIR
    )

  tuner.search(x=X_train1,y = y_train1,epochs = 20, batch_size = batch, validation_data = (X_test1,y_test1),callbacks = [stop_early])

  # Get the best hyperparameters and validation accuracy for the current batch size
  best_hyperparameters = tuner.get_best_hyperparameters()[0]
  best_val_accuracy = tuner.oracle.get_best_trials(1)[0].score

  # Store the results in the dictionary
  results100[batch] = {'best_val_accuracy': best_val_accuracy, 'best_hyperparameters': best_hyperparameters}


results100

### 200 samples

In [ ]:
# MAX_FEATURES = 6000
embed_num_dims = 300
tokenizer = Tokenizer()
tokenizer.fit_on_texts(train200['lemmatized text'])
index_of_words = tokenizer.word_index
vocab_size = len(index_of_words) + 1

tokenized_train = tokenizer.texts_to_sequences(train200['lemmatized text'])
tokenized_test = tokenizer.texts_to_sequences(test_data['lemmatized text'])


RNN_CELL_SIZE = 32

MAX_LEN = 130
X_train1 = pad_sequences(tokenized_train, maxlen=MAX_LEN,padding='pre')
y_train1 = train200['category']

X_test1 = pad_sequences(tokenized_test, maxlen=MAX_LEN,padding='pre')
y_test1 = test_data['category']



embedd_matrix = create_embedding_matrix(index_of_words, embed_num_dims)

# Different batch sizes to test
batch_sizes = (4,8,16,32,50)

# Dictionary to store validation accuracy and hyperparameters for each batch size
results200 = {}

for batch in batch_sizes:
  print('Batch size =', batch)

  LOG_DIR = f"{int(time.time())}"
  seed_value= 0
  stop_early = tf.keras.callbacks.EarlyStopping(monitor='val_accuracy',
                              patience=4,
                              restore_best_weights=True,
                              verbose=0, mode='max')
  tuner = RandomSearch(
    build_model,
    objective = keras_tuner.Objective('val_accuracy', direction="max"),
    max_trials = 10,
    executions_per_trial = 1,
    directory = LOG_DIR
    )

  tuner.search(x=X_train1,y = y_train1,epochs = 20, batch_size = batch, validation_data = (X_test1,y_test1),callbacks = [stop_early])

  # Get the best hyperparameters and validation accuracy for the current batch size
  best_hyperparameters = tuner.get_best_hyperparameters()[0]
  best_val_accuracy = tuner.oracle.get_best_trials(1)[0].score

  # Store the results in the dictionary
  results200[batch] = {'best_val_accuracy': best_val_accuracy, 'best_hyperparameters': best_hyperparameters}


results200


### 300 samples

In [ ]:
# MAX_FEATURES = 6000
embed_num_dims = 300
tokenizer = Tokenizer()
tokenizer.fit_on_texts(train300['lemmatized text'])

index_of_words = tokenizer.word_index
vocab_size = len(index_of_words) + 1

tokenized_train = tokenizer.texts_to_sequences(train300['lemmatized text'])
tokenized_test = tokenizer.texts_to_sequences(test_data['lemmatized text'])


RNN_CELL_SIZE = 32

MAX_LEN = 130
X_train1 = pad_sequences(tokenized_train, maxlen=MAX_LEN,padding='pre')
y_train1 = train300['category']

X_test1 = pad_sequences(tokenized_test, maxlen=MAX_LEN,padding='pre')
y_test1 = test_data['category']


print(X_train1.shape)
print(X_test1.shape)
print(y_train1.shape)
print(y_test1.shape)


embedd_matrix = create_embedding_matrix(index_of_words, embed_num_dims)

# Different batch sizes to test
batch_sizes = (4,8,16,32,50)

# Dictionary to store validation accuracy and hyperparameters for each batch size
results300 = {}

for batch in batch_sizes:
  print('Batch size =', batch)

  LOG_DIR = f"{int(time.time())}"
  seed_value= 0
  stop_early = tf.keras.callbacks.EarlyStopping(monitor='val_accuracy',
                              patience=4,
                              restore_best_weights=True,
                              verbose=0, mode='max')
  tuner = RandomSearch(
    build_model,
    objective = keras_tuner.Objective('val_accuracy', direction="max"),
    max_trials = 10,
    executions_per_trial = 1,
    directory = LOG_DIR
    )

  tuner.search(x=X_train1,y = y_train1,epochs = 20, batch_size = batch, validation_data = (X_test1,y_test1),callbacks = [stop_early])

  # Get the best hyperparameters and validation accuracy for the current batch size
  best_hyperparameters = tuner.get_best_hyperparameters()[0]
  best_val_accuracy = tuner.oracle.get_best_trials(1)[0].score

  # Store the results in the dictionary
  results300[batch] = {'best_val_accuracy': best_val_accuracy, 'best_hyperparameters': best_hyperparameters}


results300

### 350 samples

In [ ]:
# MAX_FEATURES = 6000
embed_num_dims = 300
tokenizer = Tokenizer()
tokenizer.fit_on_texts(train350['lemmatized text'])

index_of_words = tokenizer.word_index
vocab_size = len(index_of_words) + 1

tokenized_train = tokenizer.texts_to_sequences(train350['lemmatized text'])
tokenized_test = tokenizer.texts_to_sequences(test_data['lemmatized text'])


RNN_CELL_SIZE = 32

MAX_LEN = 130
X_train1 = pad_sequences(tokenized_train, maxlen=MAX_LEN,padding='pre')
y_train1 = train350['category']

X_test1 = pad_sequences(tokenized_test, maxlen=MAX_LEN,padding='pre')
y_test1 = test_data['category']


# X_train1, X_test1 = train_test_split(X_train1, test_size=0.2,  random_state = 42)
# y_train1, y_test1 = train_test_split(y_train1, test_size=0.2,  random_state = 42)


print(X_train1.shape)
print(X_test1.shape)
print(y_train1.shape)
print(y_test1.shape)

embedd_matrix = create_embedding_matrix(index_of_words, embed_num_dims)

# Different batch sizes to test
batch_sizes = (4,8,16,32,50)

# Dictionary to store validation accuracy and hyperparameters for each batch size
results350 = {}

for batch in batch_sizes:
  print('Batch size =', batch)

  LOG_DIR = f"{int(time.time())}"
  seed_value= 0
  stop_early = tf.keras.callbacks.EarlyStopping(monitor='val_accuracy',
                              patience=4,
                              restore_best_weights=True,
                              verbose=0, mode='max')
  tuner = RandomSearch(
    build_model,
    objective = keras_tuner.Objective('val_accuracy', direction="max"),
    max_trials = 10,
    executions_per_trial = 1,
    directory = LOG_DIR
    )

  tuner.search(x=X_train1,y = y_train1,epochs = 20, batch_size = batch, validation_data = (X_test1,y_test1),callbacks = [stop_early])

  # Get the best hyperparameters and validation accuracy for the current batch size
  best_hyperparameters = tuner.get_best_hyperparameters()[0]
  best_val_accuracy = tuner.oracle.get_best_trials(1)[0].score

  # Store the results in the dictionary
  results350[batch] = {'best_val_accuracy': best_val_accuracy, 'best_hyperparameters': best_hyperparameters}


results350

### 400 samples

In [ ]:
# MAX_FEATURES = 6000
embed_num_dims = 300
tokenizer = Tokenizer()
tokenizer.fit_on_texts(train400['lemmatized text'])

index_of_words = tokenizer.word_index
vocab_size = len(index_of_words) + 1

tokenized_train = tokenizer.texts_to_sequences(train400['lemmatized text'])
tokenized_test = tokenizer.texts_to_sequences(test_data['lemmatized text'])


RNN_CELL_SIZE = 32

MAX_LEN = 130
X_train1 = pad_sequences(tokenized_train, maxlen=MAX_LEN,padding='pre')
y_train1 = train400['category']

X_test1 = pad_sequences(tokenized_test, maxlen=MAX_LEN,padding='pre')
y_test1 = test_data['category']


print(X_train1.shape)
print(X_test1.shape)
print(y_train1.shape)
print(y_test1.shape)

embedd_matrix = create_embedding_matrix(index_of_words, embed_num_dims)

# Different batch sizes to test
batch_sizes = (4,8,16,32,50)

# Dictionary to store validation accuracy and hyperparameters for each batch size
results400 = {}

for batch in batch_sizes:
  print('Batch size =', batch)

  LOG_DIR = f"{int(time.time())}"
  seed_value= 0
  stop_early = tf.keras.callbacks.EarlyStopping(monitor='val_accuracy',
                              patience=4,
                              restore_best_weights=True,
                              verbose=0, mode='max')
  tuner = RandomSearch(
    build_model,
    objective = keras_tuner.Objective('val_accuracy', direction="max"),
    max_trials = 10,
    executions_per_trial = 1,
    directory = LOG_DIR
    )

  tuner.search(x=X_train1,y = y_train1,epochs = 20, batch_size = batch, validation_data = (X_test1,y_test1),callbacks = [stop_early])

  # Get the best hyperparameters and validation accuracy for the current batch size
  best_hyperparameters = tuner.get_best_hyperparameters()[0]
  best_val_accuracy = tuner.oracle.get_best_trials(1)[0].score

  # Store the results in the dictionary
  results400[batch] = {'best_val_accuracy': best_val_accuracy, 'best_hyperparameters': best_hyperparameters}


results400

### 450 samples

In [ ]:
# MAX_FEATURES = 6000
embed_num_dims = 300
tokenizer = Tokenizer()
tokenizer.fit_on_texts(train450['lemmatized text'])

index_of_words = tokenizer.word_index
vocab_size = len(index_of_words) + 1

tokenized_train = tokenizer.texts_to_sequences(train450['lemmatized text'])
tokenized_test = tokenizer.texts_to_sequences(test_data['lemmatized text'])


RNN_CELL_SIZE = 32

MAX_LEN = 130
X_train1 = pad_sequences(tokenized_train, maxlen=MAX_LEN,padding='pre')
y_train1 = train450['category']

X_test1 = pad_sequences(tokenized_test, maxlen=MAX_LEN,padding='pre')
y_test1 = test_data['category']


print(X_train1.shape)
print(X_test1.shape)
print(y_train1.shape)
print(y_test1.shape)

embedd_matrix = create_embedding_matrix(index_of_words, embed_num_dims)

# Different batch sizes to test
batch_sizes = (4,8,16,32,50)

# Dictionary to store validation accuracy and hyperparameters for each batch size
results450 = {}

for batch in batch_sizes:
  print('Batch size =', batch)

  LOG_DIR = f"{int(time.time())}"
  seed_value= 0
  stop_early = tf.keras.callbacks.EarlyStopping(monitor='val_accuracy',
                              patience=4,
                              restore_best_weights=True,
                              verbose=0, mode='max')
  tuner = RandomSearch(
    build_model,
    objective = keras_tuner.Objective('val_accuracy', direction="max"),
    max_trials = 10,
    executions_per_trial = 1,
    directory = LOG_DIR
    )

  tuner.search(x=X_train1,y = y_train1,epochs = 20, batch_size = batch, validation_data = (X_test1,y_test1),callbacks = [stop_early])

  # Get the best hyperparameters and validation accuracy for the current batch size
  best_hyperparameters = tuner.get_best_hyperparameters()[0]
  best_val_accuracy = tuner.oracle.get_best_trials(1)[0].score

  # Store the results in the dictionary
  results450[batch] = {'best_val_accuracy': best_val_accuracy, 'best_hyperparameters': best_hyperparameters}


results450

### Hyperparameters of the best-performing model

In [ ]:
# Access the hyperparameters of the best-perfroming model based on the batch size

best_hyperparameters = results[8]['best_hyperparameters']

print('cnn1 size =', best_hyperparameters.get('cnn_1_unit'))
print('cnn1 dropout =', best_hyperparameters.get('cnn_1_dropout'))
print('attention vector =', best_hyperparameters.get('attention_unit'))
print('bilstm size =', best_hyperparameters.get('lstm_unit'))
print('bilstm dropout =', best_hyperparameters.get('lstm_dropout'))
print('cnn2 size =', best_hyperparameters.get('cnn_2_unit'))
print('cnn2 dropout =', best_hyperparameters.get('cnn_2_dropout'))
print('learning rate =', best_hyperparameters.get('learning rate'))


## Similarity filtering and Data augmentation

Train the model with the filtered and augmented data to see if there will be a performance improvement. Use the layer sizes, dropout ratios and batch size which resulted in the highest accuracy in the previous section.

The baseline model compiled with the best-performing layer sizes, dropout ratios, learning rate and batch size (according to the previous section) and optimized with **RMSprop optimizer**

In [ ]:
def adjusted_baseline(hp):

  reproduceResult()

  # INPUT DEFINITION
  seq_input = keras.layers.Input(shape=(MAX_LEN,))

  # EMBEDDING
  embedded = keras.layers.Embedding(vocab_size,
                          embed_num_dims,
                          input_length = MAX_LEN,
                          weights = [embedd_matrix])(seq_input)

  # 1ST CNN LAYER
  cnn_1 = keras.layers.Conv1D(16,3)(embedded)
  cnn_1 = keras.layers.Activation(activation='relu')(cnn_1)
  cnn_1 = keras.layers.BatchNormalization()(cnn_1)
  cnn_1 = keras.layers.Dropout(0.1,seed=seed_value)(cnn_1)

  # ATTENTION
  attention = keras.layers.TimeDistributed(keras.layers.Dense(112))(cnn_1)
  attention = keras.layers.Reshape((128,112))(attention)
  attention = keras.layers.Activation('relu', name = 'cnn1_attention')(attention)
  attention_output = keras.layers.Dot(axes = 1)([cnn_1, attention])

  # BILSTM LAYER
  lstm = keras.layers.Bidirectional(keras.layers.LSTM(224,
                                                      return_sequences=True,input_shape =(128,)))(attention_output)
  lstm = keras.layers.Activation(activation='relu')(lstm)
  lstm = keras.layers.BatchNormalization()(lstm)
  lstm = keras.layers.Dropout(0.1,seed=seed_value)(lstm)

  # 2ND CNN LAYER
  cnn_2 = keras.layers.Conv1D(192,3)(lstm)
  cnn_2 = keras.layers.Activation(activation='relu')(cnn_2)
  cnn_2 = keras.layers.BatchNormalization()(cnn_2)
  cnn_2 = keras.layers.Dropout(0.5,seed=seed_value)(cnn_2)

  max_pooling = keras.layers.GlobalMaxPooling1D()(cnn_2)
  output = keras.layers.Dense(6, activation='softmax')(max_pooling)

  model = keras.Model(inputs = [seq_input], outputs = output)
  model.compile(loss='sparse_categorical_crossentropy', optimizer = keras.optimizers.RMSprop(learning_rate = 0.00035000000000000004), metrics=['accuracy'])
  # use the learning rate value from the hyperparameters of the best-perfroming model

  return model

The same model optimized with **ADAM optimizer**

In [ ]:
def adjusted_baseline_adam(hp):

  reproduceResult()

  # INPUT DEFINITION
  seq_input = keras.layers.Input(shape=(MAX_LEN,))

  # EMBEDDING
  embedded = keras.layers.Embedding(vocab_size,
                          embed_num_dims,
                          input_length = MAX_LEN,
                          weights = [embedd_matrix])(seq_input)

  # 1ST CNN LAYER
  cnn_1 = keras.layers.Conv1D(16,3)(embedded)
  cnn_1 = keras.layers.Activation(activation='relu')(cnn_1)
  cnn_1 = keras.layers.BatchNormalization()(cnn_1)
  cnn_1 = keras.layers.Dropout(0.1,seed=seed_value)(cnn_1)

  # ATTENTION
  attention = keras.layers.TimeDistributed(keras.layers.Dense(112))(cnn_1)
  attention = keras.layers.Reshape((128,112))(attention)
  attention = keras.layers.Activation('relu', name = 'cnn1_attention')(attention)
  attention_output = keras.layers.Dot(axes = 1)([cnn_1, attention])

  # BILSTM LAYER
  lstm = keras.layers.Bidirectional(keras.layers.LSTM(224,
                                                      return_sequences=True,input_shape =(128,)))(attention_output)
  lstm = keras.layers.Activation(activation='relu')(lstm)
  lstm = keras.layers.BatchNormalization()(lstm)
  lstm = keras.layers.Dropout(0.1,seed=seed_value)(lstm)

  # 2ND CNN LAYER
  cnn_2 = keras.layers.Conv1D(192,3)(lstm)
  cnn_2 = keras.layers.Activation(activation='relu')(cnn_2)
  cnn_2 = keras.layers.BatchNormalization()(cnn_2)
  cnn_2 = keras.layers.Dropout(0.5,seed=seed_value)(cnn_2)

  max_pooling = keras.layers.GlobalMaxPooling1D()(cnn_2)
  output = keras.layers.Dense(6, activation='softmax')(max_pooling)

  model = keras.Model(inputs = [seq_input], outputs = output)
  model.compile(loss='sparse_categorical_crossentropy', optimizer = keras.optimizers.Adam(learning_rate = 0.00035000000000000004), metrics=['accuracy'])

  return model

### Original train data with Adam

In [ ]:
# MAX_FEATURES = 6000
embed_num_dims = 300
tokenizer = Tokenizer()
tokenizer.fit_on_texts(train_data['lemmatized text'])
index_of_words = tokenizer.word_index
vocab_size = len(index_of_words) + 1

tokenized_train = tokenizer.texts_to_sequences(train_data['lemmatized text'])

RNN_CELL_SIZE = 32

MAX_LEN = 130
X_train1 = pad_sequences(tokenized_train, maxlen=MAX_LEN,padding='pre')
y_train1 = train_data['category']


embedd_matrix = create_embedding_matrix(index_of_words, embed_num_dims)

LOG_DIR = f"{int(time.time())}"
seed_value= 0

# Adam

tuner = RandomSearch(
    adjusted_baseline_adam,
    objective = keras_tuner.Objective('val_accuracy', direction="max"),
    max_trials = 10,
    executions_per_trial = 1,
    directory = LOG_DIR
    )

tuner.search(x=X_train1,y = y_train1,epochs = 20, batch_size = 8, validation_data = (X_test1,y_test1),callbacks = [stop_early])


tuner.results_summary()

In [ ]:
LOG_DIR = f"{int(time.time())}"
seed_value= 0

# RMSprop

tuner = RandomSearch(
    adjusted_baseline,
    objective = keras_tuner.Objective('val_accuracy', direction="max"),
    max_trials = 10,
    executions_per_trial = 1,
    directory = LOG_DIR
    )

tuner.search(x=X_train1,y = y_train1,epochs = 20, batch_size = 8, validation_data = (X_test1,y_test1),callbacks = [stop_early])


tuner.results_summary()

### filtered 150

In [ ]:
filtered_150['lemmatized text'] = filtered_150.text.apply(lambda x: lemmatization(x))

# MAX_FEATURES = 6000
embed_num_dims = 300
tokenizer = Tokenizer()
tokenizer.fit_on_texts(filtered_150['lemmatized text'])
index_of_words = tokenizer.word_index
vocab_size = len(index_of_words) + 1

tokenized_train = tokenizer.texts_to_sequences(filtered_150['lemmatized text'])

RNN_CELL_SIZE = 32

MAX_LEN = 130
X_train1 = pad_sequences(tokenized_train, maxlen=MAX_LEN,padding='pre')
y_train1 = filtered_150['category']


print(X_train1.shape)
print(y_train1.shape)

embedd_matrix = create_embedding_matrix(index_of_words, embed_num_dims)


In [ ]:
LOG_DIR = f"{int(time.time())}"
seed_value= 0

# RMSprop

tuner = RandomSearch(
    adjusted_baseline,
    objective = keras_tuner.Objective('val_accuracy', direction="max"),
    max_trials = 10,
    executions_per_trial = 1,
    directory = LOG_DIR
    )

tuner.search(x=X_train1,y = y_train1,epochs = 20, batch_size = 8, validation_data = (X_test1,y_test1),callbacks = [stop_early])


tuner.results_summary()

In [ ]:
LOG_DIR = f"{int(time.time())}"
seed_value= 0

# Adam

tuner = RandomSearch(
    adjusted_baseline_adam,
    objective = keras_tuner.Objective('val_accuracy', direction="max"),
    max_trials = 10,
    executions_per_trial = 1,
    directory = LOG_DIR
    )

tuner.search(x=X_train1,y = y_train1,epochs = 20, batch_size = 8, validation_data = (X_test1,y_test1),callbacks = [stop_early])


tuner.results_summary()

### filtered & augmented 150

In [ ]:
aug_train150['lemmatized text'] = aug_train150.text.apply(lambda x: lemmatization(x))

# MAX_FEATURES = 6000
embed_num_dims = 300
tokenizer = Tokenizer()
tokenizer.fit_on_texts(aug_train150['lemmatized text'])
index_of_words = tokenizer.word_index
vocab_size = len(index_of_words) + 1

tokenized_train = tokenizer.texts_to_sequences(aug_train150['lemmatized text'])

RNN_CELL_SIZE = 32

MAX_LEN = 130
X_train1 = pad_sequences(tokenized_train, maxlen=MAX_LEN,padding='pre')
y_train1 = aug_train150['category']


embedd_matrix = create_embedding_matrix(index_of_words, embed_num_dims)


In [ ]:
LOG_DIR = f"{int(time.time())}"
seed_value= 0

# RMSprop


tuner = RandomSearch(
    adjusted_baseline,
    objective = keras_tuner.Objective('val_accuracy', direction="max"),
    max_trials = 10,
    executions_per_trial = 1,
    directory = LOG_DIR
    )

tuner.search(x=X_train1,y = y_train1,epochs = 20, batch_size = 8, validation_data = (X_test1,y_test1),callbacks = [stop_early])


tuner.results_summary()

In [ ]:
LOG_DIR = f"{int(time.time())}"
seed_value= 0

# Adam


tuner = RandomSearch(
    adjusted_baseline_adam,
    objective = keras_tuner.Objective('val_accuracy', direction="max"),
    max_trials = 10,
    executions_per_trial = 1,
    directory = LOG_DIR
    )

tuner.search(x=X_train1,y = y_train1,epochs = 20, batch_size = 8, validation_data = (X_test1,y_test1),callbacks = [stop_early])


tuner.results_summary()

### filtered 312

In [ ]:

filtered_312['lemmatized text'] = filtered_312.text.apply(lambda x: lemmatization(x))

# MAX_FEATURES = 6000
embed_num_dims = 300
tokenizer = Tokenizer()
tokenizer.fit_on_texts(filtered_312['lemmatized text'])
index_of_words = tokenizer.word_index
vocab_size = len(index_of_words) + 1

tokenized_train = tokenizer.texts_to_sequences(filtered_312['lemmatized text'])

RNN_CELL_SIZE = 32

MAX_LEN = 130
X_train1 = pad_sequences(tokenized_train, maxlen=MAX_LEN,padding='pre')
y_train1 = filtered_312['category']


embedd_matrix = create_embedding_matrix(index_of_words, embed_num_dims)



In [ ]:
LOG_DIR = f"{int(time.time())}"
seed_value= 0

# RMSprop


tuner = RandomSearch(
    adjusted_baseline,
    objective = keras_tuner.Objective('val_accuracy', direction="max"),
    max_trials = 10,
    executions_per_trial = 1,
    directory = LOG_DIR
    )

tuner.search(x=X_train1,y = y_train1,epochs = 20, batch_size = 8, validation_data = (X_test1,y_test1),callbacks = [stop_early])


tuner.results_summary()

In [ ]:
LOG_DIR = f"{int(time.time())}"
seed_value= 0

# Adam


tuner = RandomSearch(
    adjusted_baseline_adam,
    objective = keras_tuner.Objective('val_accuracy', direction="max"),
    max_trials = 10,
    executions_per_trial = 1,
    directory = LOG_DIR
    )

tuner.search(x=X_train1,y = y_train1,epochs = 20, batch_size = 8, validation_data = (X_test1,y_test1),callbacks = [stop_early])


tuner.results_summary()

### filtered & augmented 312

In [ ]:
aug_train312['lemmatized text'] = aug_train312.text.apply(lambda x: lemmatization(x))

# MAX_FEATURES = 6000
embed_num_dims = 300
tokenizer = Tokenizer()
tokenizer.fit_on_texts(aug_train312['lemmatized text'])
index_of_words = tokenizer.word_index
vocab_size = len(index_of_words) + 1

tokenized_train = tokenizer.texts_to_sequences(aug_train312['lemmatized text'])

RNN_CELL_SIZE = 32

MAX_LEN = 130
X_train1 = pad_sequences(tokenized_train, maxlen=MAX_LEN,padding='pre')
y_train1 = aug_train312['category']


embedd_matrix = create_embedding_matrix(index_of_words, embed_num_dims)


In [ ]:
LOG_DIR = f"{int(time.time())}"
seed_value= 0

# RMSprop


tuner = RandomSearch(
    adjusted_baseline,
    objective = keras_tuner.Objective('val_accuracy', direction="max"),
    max_trials = 10,
    executions_per_trial = 1,
    directory = LOG_DIR
    )

tuner.search(x=X_train1,y = y_train1,epochs = 20, batch_size = 8, validation_data = (X_test1,y_test1),callbacks = [stop_early])


tuner.results_summary()

In [ ]:
LOG_DIR = f"{int(time.time())}"
seed_value= 0

# Adam


tuner = RandomSearch(
    adjusted_baseline_adam,
    objective = keras_tuner.Objective('val_accuracy', direction="max"),
    max_trials = 10,
    executions_per_trial = 1,
    directory = LOG_DIR
    )

tuner.search(x=X_train1,y = y_train1,epochs = 20, batch_size = 8, validation_data = (X_test1,y_test1),callbacks = [stop_early])


tuner.results_summary()

## Fusion architectures

### CNN + att + BiLSTM

In [ ]:
from tensorflow.keras import regularizers
import time
LOG_DIR = f"{int(time.time())}"
seed_value= 0

def build_model(hp):

  reproduceResult()

  print('Ya it comes here')

  # SEARCH VALUES
  att_size = hp.Int("attention size",min_value =16, max_value = 128, step = 16)
  cnn_1_size = hp.Int("cnn1 size",min_value =16, max_value = 96, step = 16)
  cnn_1_dropout = hp.Float("cnn1 dropout",min_value = 0.1,max_value = 0.3,step = 0.1)

  lstm_size = hp.Int("lstm size",min_value =32, max_value = 256, step = 32)
  lstm_dropout = hp.Float("lstm dropout",min_value = 0.1,max_value = 0.5,step = 0.1)
  cnn_2_size = hp.Int("cnn2 size",min_value =32, max_value = 256, step = 32)
  cnn_2_dropout = hp.Float("cnn2 dropout",min_value = 0.1,max_value = 0.5,step = 0.1)

  lr = hp.Float("learning rate",min_value = 2e-5 ,max_value = 1e-3,step = 2e-5)


  # INPUT DEFINITION
  seq_input = keras.layers.Input(shape=(MAX_LEN,))

  # EMBEDDING
  embedded = keras.layers.Embedding(vocab_size,
                          embed_num_dims,
                          input_length = MAX_LEN,
                          weights = [embedd_matrix])(seq_input)

  # 1ST CNN LAYER
  cnn_1 = keras.layers.Conv1D(cnn_1_size,3)(embedded)
  cnn_1 = keras.layers.Activation(activation='relu')(cnn_1)
  cnn_1 = keras.layers.BatchNormalization()(cnn_1)
  cnn_1 = keras.layers.Dropout(cnn_1_dropout,seed=seed_value)(cnn_1)

  # ATTENTION
  attention = keras.layers.TimeDistributed(keras.layers.Dense(att_size))(cnn_1)
  attention = keras.layers.Reshape((128,att_size))(attention)
  attention = keras.layers.Activation('relu', name = 'cnn1_attention')(attention)
  attention_output = keras.layers.Dot(axes = 1)([cnn_1, attention])

  # BILSTM LAYER
  lstm = keras.layers.Bidirectional(keras.layers.LSTM(lstm_size,
                                                      return_sequences=True,input_shape =(128,)))(attention_output)
  lstm = keras.layers.Activation(activation='relu')(lstm)
  lstm = keras.layers.BatchNormalization()(lstm)
  lstm = keras.layers.Dropout(lstm_dropout,seed=seed_value)(lstm)

  max_pooling = keras.layers.GlobalMaxPooling1D()(lstm)
  output = keras.layers.Dense(6, activation='softmax')(max_pooling)

  model = keras.Model(inputs = [seq_input], outputs = output)
  model.compile(loss='sparse_categorical_crossentropy', optimizer=keras.optimizers.Adam(learning_rate=lr), metrics=['accuracy'])

  return model


# EARLY STOPPING
stop_early = tf.keras.callbacks.EarlyStopping(monitor='val_accuracy',
                              patience=4,
                              restore_best_weights=True,
                              verbose=0, mode='max')

# PARAMETER SEARCH
tuner = RandomSearch(
    build_model,
    objective = keras_tuner.Objective('val_accuracy', direction="max"),
    max_trials = 10,
    executions_per_trial = 1,
    directory = LOG_DIR
    )

tuner.search(x=X_train1,y = y_train1,epochs = 20, batch_size = 8, validation_data = (X_test1,y_test1),callbacks = [stop_early])


tuner.results_summary()

### CNN + att + BiLSTM + CNN

In [ ]:
LOG_DIR = f"{int(time.time())}"
seed_value= 0

def build_model(hp):

  reproduceResult()

  print('Ya it comes here')

  # SEARCH VALUES
  att_size = hp.Int("attention size",min_value =16, max_value = 128, step = 16)
  cnn_1_size = hp.Int("cnn1 size",min_value =16, max_value = 96, step = 16)
  cnn_1_dropout = hp.Float("cnn1 dropout",min_value = 0.1,max_value = 0.3,step = 0.1)

  lstm_size = hp.Int("lstm size",min_value =32, max_value = 256, step = 32)
  lstm_dropout = hp.Float("lstm dropout",min_value = 0.1,max_value = 0.5,step = 0.1)
  cnn_2_size = hp.Int("cnn2 size",min_value =32, max_value = 256, step = 32)
  cnn_2_dropout = hp.Float("cnn2 dropout",min_value = 0.1,max_value = 0.5,step = 0.1)

  lr = hp.Float("learning rate",min_value = 2e-5 ,max_value = 1e-3,step = 2e-5)


  # INPUT DEFINITION
  seq_input = keras.layers.Input(shape=(MAX_LEN,))

  # EMBEDDING
  embedded = keras.layers.Embedding(vocab_size,
                          embed_num_dims,
                          input_length = MAX_LEN,
                          weights = [embedd_matrix])(seq_input)

  # 1ST CNN LAYER
  cnn_1 = keras.layers.Conv1D(cnn_1_size,3)(embedded)
  cnn_1 = keras.layers.Activation(activation='relu')(cnn_1)
  cnn_1 = keras.layers.BatchNormalization()(cnn_1)
  cnn_1 = keras.layers.Dropout(cnn_1_dropout,seed=seed_value)(cnn_1)

  # ATTENTION
  attention = keras.layers.TimeDistributed(keras.layers.Dense(att_size))(cnn_1)
  attention = keras.layers.Reshape((128,att_size))(attention)
  attention = keras.layers.Activation('relu', name = 'cnn1_attention')(attention)
  attention_output = keras.layers.Dot(axes = 1)([cnn_1, attention])

  # BILSTM LAYER
  lstm = keras.layers.Bidirectional(keras.layers.LSTM(lstm_size,
                                                      return_sequences=True,input_shape =(128,)))(attention_output)
  lstm = keras.layers.Activation(activation='relu')(lstm)
  lstm = keras.layers.BatchNormalization()(lstm)
  lstm = keras.layers.Dropout(lstm_dropout,seed=seed_value)(lstm)

  # 2ND CNN LAYER
  cnn_2 = keras.layers.Conv1D(cnn_2_size,3)(lstm)
  cnn_2 = keras.layers.Activation(activation='relu')(cnn_2)
  cnn_2 = keras.layers.BatchNormalization()(cnn_2)
  cnn_2 = keras.layers.Dropout(cnn_2_dropout,seed=seed_value)(cnn_2)

  max_pooling = keras.layers.GlobalMaxPooling1D()(cnn_2)
  output = keras.layers.Dense(6, activation='softmax')(max_pooling)

  model = keras.Model(inputs = [seq_input], outputs = output)
  model.compile(loss='sparse_categorical_crossentropy', optimizer=keras.optimizers.Adam(learning_rate=lr), metrics=['accuracy'])

  return model


# EARLY STOPPING
stop_early = tf.keras.callbacks.EarlyStopping(monitor='val_accuracy',
                              patience=4,
                              restore_best_weights=True,
                              verbose=0, mode='max')

# PARAMETER SEARCH
tuner = RandomSearch(
    build_model,
    objective = keras_tuner.Objective('val_accuracy', direction="max"),
    max_trials = 10,
    executions_per_trial = 1,
    directory = LOG_DIR
    )

tuner.search(x=X_train1,y = y_train1,epochs = 20, batch_size = 8, validation_data = (X_test1,y_test1),callbacks = [stop_early])


tuner.results_summary()

### CNN + att + BiLSTM + CNN + CNN

In [ ]:
LOG_DIR = f"{int(time.time())}"
seed_value= 0

def build_model(hp):

  reproduceResult()

  print('Ya it comes here')

  # SEARCH VALUES
  att_size = hp.Int("attention size",min_value =16, max_value = 128, step = 16)
  cnn_1_size = hp.Int("cnn1 size",min_value =16, max_value = 96, step = 16)
  cnn_1_dropout = hp.Float("cnn1 dropout",min_value = 0.1,max_value = 0.3,step = 0.1)

  lstm_size = hp.Int("lstm size",min_value =32, max_value = 256, step = 32)
  lstm_dropout = hp.Float("lstm dropout",min_value = 0.1,max_value = 0.5,step = 0.1)
  cnn_2_size = hp.Int("cnn2 size",min_value =32, max_value = 256, step = 32)
  cnn_2_dropout = hp.Float("cnn2 dropout",min_value = 0.1,max_value = 0.5,step = 0.1)

  cnn_3_size = hp.Int("cnn3 size",min_value =32, max_value = 256, step = 32)
  cnn_3_dropout = hp.Float("cnn3 dropout",min_value = 0.1,max_value = 0.5,step = 0.1)

  lr = hp.Float("learning rate",min_value = 2e-5 ,max_value = 1e-3,step = 2e-5)


  # INPUT DEFINITION
  seq_input = keras.layers.Input(shape=(MAX_LEN,))

  # EMBEDDING
  embedded = keras.layers.Embedding(vocab_size,
                          embed_num_dims,
                          input_length = MAX_LEN,
                          weights = [embedd_matrix])(seq_input)

  # 1ST CNN LAYER
  cnn_1 = keras.layers.Conv1D(cnn_1_size,3)(embedded)
  cnn_1 = keras.layers.Activation(activation='relu')(cnn_1)
  cnn_1 = keras.layers.BatchNormalization()(cnn_1)
  cnn_1 = keras.layers.Dropout(cnn_1_dropout,seed=seed_value)(cnn_1)

  # ATTENTION
  attention = keras.layers.TimeDistributed(keras.layers.Dense(att_size))(cnn_1)
  attention = keras.layers.Reshape((128,att_size))(attention)
  attention = keras.layers.Activation('relu', name = 'cnn1_attention')(attention)
  attention_output = keras.layers.Dot(axes = 1)([cnn_1, attention])

  # BILSTM LAYER
  lstm = keras.layers.Bidirectional(keras.layers.LSTM(lstm_size,
                                                      return_sequences=True,input_shape =(128,)))(attention_output)
  lstm = keras.layers.Activation(activation='relu')(lstm)
  lstm = keras.layers.BatchNormalization()(lstm)
  lstm = keras.layers.Dropout(lstm_dropout,seed=seed_value)(lstm)

  # 2ND CNN LAYER
  cnn_2 = keras.layers.Conv1D(cnn_2_size,3)(lstm)
  cnn_2 = keras.layers.Activation(activation='relu')(cnn_2)
  cnn_2 = keras.layers.BatchNormalization()(cnn_2)
  cnn_2 = keras.layers.Dropout(cnn_2_dropout,seed=seed_value)(cnn_2)

  # 3ND CNN LAYER
  cnn_3 = keras.layers.Conv1D(cnn_3_size,3)(lstm)
  cnn_3 = keras.layers.Activation(activation='relu')(cnn_3)
  cnn_3 = keras.layers.BatchNormalization()(cnn_3)
  cnn_3 = keras.layers.Dropout(cnn_2_dropout,seed=seed_value)(cnn_3)

  max_pooling = keras.layers.GlobalMaxPooling1D()(cnn_3)
  output = keras.layers.Dense(6, activation='softmax')(max_pooling)

  model = keras.Model(inputs = [seq_input], outputs = output)
  model.compile(loss='sparse_categorical_crossentropy', optimizer=keras.optimizers.Adam(learning_rate=lr), metrics=['accuracy'])

  return model


# EARLY STOPPING
stop_early = tf.keras.callbacks.EarlyStopping(monitor='val_accuracy',
                              patience=4,
                              restore_best_weights=True,
                              verbose=0, mode='max')

# PARAMETER SEARCH
tuner = RandomSearch(
    build_model,
    objective = keras_tuner.Objective('val_accuracy', direction="max"),
    max_trials = 10,
    executions_per_trial = 1,
    directory = LOG_DIR
    )

tuner.search(x=X_train1,y = y_train1,epochs = 20, batch_size = 8, validation_data = (X_test1,y_test1),callbacks = [stop_early])


tuner.results_summary()

### CNN + A + BiLSTM + CNN + BiLSTM

In [ ]:
LOG_DIR = f"{int(time.time())}"
seed_value= 0

def build_model(hp):

  reproduceResult()

  print('Ya it comes here')

  # SEARCH VALUES
  att_size = hp.Int("attention size",min_value =16, max_value = 128, step = 16)
  cnn_1_size = hp.Int("cnn1 size",min_value =16, max_value = 96, step = 16)
  cnn_1_dropout = hp.Float("cnn1 dropout",min_value = 0.1,max_value = 0.3,step = 0.1)

  lstm_1_size = hp.Int("lstm1 size",min_value =32, max_value = 256, step = 32)
  lstm_1_dropout = hp.Float("lstm1 dropout",min_value = 0.1,max_value = 0.5,step = 0.1)

  cnn_2_size = hp.Int("cnn2 size",min_value =32, max_value = 256, step = 32)
  cnn_2_dropout = hp.Float("cnn2 dropout",min_value = 0.1,max_value = 0.5,step = 0.1)

  lstm_2_size = hp.Int("lstm2 size",min_value =32, max_value = 256, step = 32)
  lstm_2_dropout = hp.Float("lstm2 dropout",min_value = 0.1,max_value = 0.5,step = 0.1)

  lr = hp.Float("learning rate",min_value = 2e-5 ,max_value = 1e-3,step = 2e-5)


  # INPUT DEFINITION
  seq_input = keras.layers.Input(shape=(MAX_LEN,))

  # EMBEDDING
  embedded = keras.layers.Embedding(vocab_size,
                          embed_num_dims,
                          input_length = MAX_LEN,
                          weights = [embedd_matrix])(seq_input)

  # 1ST CNN LAYER
  cnn_1 = keras.layers.Conv1D(cnn_1_size,3)(embedded)
  cnn_1 = keras.layers.Activation(activation='relu')(cnn_1)
  cnn_1 = keras.layers.BatchNormalization()(cnn_1)
  cnn_1 = keras.layers.Dropout(cnn_1_dropout,seed=seed_value)(cnn_1)

  # ATTENTION
  attention = keras.layers.TimeDistributed(keras.layers.Dense(att_size))(cnn_1)
  attention = keras.layers.Reshape((128,att_size))(attention)
  attention = keras.layers.Activation('relu', name = 'cnn1_attention')(attention)
  attention_output = keras.layers.Dot(axes = 1)([cnn_1, attention])

  # 1ST BILSTM LAYER
  lstm_1 = keras.layers.Bidirectional(keras.layers.LSTM(lstm_1_size,
                                                      return_sequences=True,input_shape =(128,)))(attention_output)
  lstm_1 = keras.layers.Activation(activation='relu')(lstm_1)
  lstm_1 = keras.layers.BatchNormalization()(lstm_1)
  lstm_1 = keras.layers.Dropout(lstm_1_dropout,seed=seed_value)(lstm_1)

  # 2ND CNN LAYER
  cnn_2 = keras.layers.Conv1D(cnn_2_size,3)(lstm_1)
  cnn_2 = keras.layers.Activation(activation='relu')(cnn_2)
  cnn_2 = keras.layers.BatchNormalization()(cnn_2)
  cnn_2 = keras.layers.Dropout(cnn_2_dropout,seed=seed_value)(cnn_2)

  # 2ND BILSTM LAYER
  lstm_2 = keras.layers.Bidirectional(keras.layers.LSTM(lstm_2_size,
                                                      return_sequences=True,input_shape =(128,)))(attention_output)
  lstm_2 = keras.layers.Activation(activation='relu')(lstm_2)
  lstm_2 = keras.layers.BatchNormalization()(lstm_2)
  lstm_2 = keras.layers.Dropout(lstm_2_dropout,seed=seed_value)(lstm_2)


  max_pooling = keras.layers.GlobalMaxPooling1D()(lstm_2)
  output = keras.layers.Dense(6, activation='softmax')(max_pooling)

  model = keras.Model(inputs = [seq_input], outputs = output)
  model.compile(loss='sparse_categorical_crossentropy', optimizer=keras.optimizers.Adam(learning_rate=lr), metrics=['accuracy'])

  return model


# EARLY STOPPING
stop_early = tf.keras.callbacks.EarlyStopping(monitor='val_accuracy',
                              patience=4,
                              restore_best_weights=True,
                              verbose=0, mode='max')

# PARAMETER SEARCH
tuner = RandomSearch(
    build_model,
    objective = keras_tuner.Objective('val_accuracy', direction="max"),
    max_trials = 10,
    executions_per_trial = 1,
    directory = LOG_DIR
    )

tuner.search(x=X_train1,y = y_train1,epochs = 20, batch_size = 8, validation_data = (X_test1,y_test1),callbacks = [stop_early])


tuner.results_summary()

### CNN + att + LSTM

In [ ]:
LOG_DIR = f"{int(time.time())}"
seed_value= 0

def build_model(hp):

  reproduceResult()

  print('Ya it comes here')

  # SEARCH VALUES
  att_size = hp.Int("attention size",min_value =16, max_value = 128, step = 16)
  cnn_1_size = hp.Int("cnn1 size",min_value =16, max_value = 96, step = 16)
  cnn_1_dropout = hp.Float("cnn1 dropout",min_value = 0.1,max_value = 0.3,step = 0.1)

  lstm_size = hp.Int("lstm size",min_value =32, max_value = 256, step = 32)
  lstm_dropout = hp.Float("lstm dropout",min_value = 0.1,max_value = 0.5,step = 0.1)


  lr = hp.Float("learning rate",min_value = 2e-5 ,max_value = 1e-3,step = 2e-5)


  # INPUT DEFINITION
  seq_input = keras.layers.Input(shape=(MAX_LEN,))

  # EMBEDDING
  embedded = keras.layers.Embedding(vocab_size,
                          embed_num_dims,
                          input_length = MAX_LEN,
                          weights = [embedd_matrix])(seq_input)


  # 1ST CNN LAYER
  cnn_1 = keras.layers.Conv1D(cnn_1_size,3)(embedded)
  cnn_1 = keras.layers.Activation(activation='relu')(cnn_1)
  cnn_1 = keras.layers.BatchNormalization()(cnn_1)
  cnn_1 = keras.layers.Dropout(cnn_1_dropout,seed=seed_value)(cnn_1)

  # ATTENTION
  attention = keras.layers.TimeDistributed(keras.layers.Dense(att_size))(cnn_1)
  attention = keras.layers.Reshape((128,att_size))(attention)
  attention = keras.layers.Activation('relu', name = 'cnn1_attention')(attention)
  attention_output = keras.layers.Dot(axes = 1)([cnn_1, attention])

  # LSTM LAYER
  lstm = keras.layers.LSTM(lstm_size,return_sequences=True,input_shape =(128,))(attention_output)
  lstm = keras.layers.Activation(activation='relu')(lstm)
  lstm = keras.layers.BatchNormalization()(lstm)
  lstm = keras.layers.Dropout(lstm_dropout,seed=seed_value)(lstm)

  max_pooling = keras.layers.GlobalMaxPooling1D()(lstm)
  output = keras.layers.Dense(6, activation='softmax')(max_pooling)

  model = keras.Model(inputs = [seq_input], outputs = output)
  model.compile(loss='sparse_categorical_crossentropy', optimizer=keras.optimizers.Adam(learning_rate=lr), metrics=['accuracy'])

  return model

# EARLY STOPPING
stop_early = tf.keras.callbacks.EarlyStopping(monitor='val_accuracy',
                              patience=4,
                              restore_best_weights=True,
                              verbose=0, mode='max')


# PARAMETER SEARCH
tuner = RandomSearch(
    build_model,
    objective = keras_tuner.Objective('val_accuracy', direction="max"),
    max_trials = 10,
    executions_per_trial = 1,
    directory = LOG_DIR
    )

tuner.search(x=X_train1,y = y_train1,epochs = 20, batch_size = 8, validation_data = (X_test1,y_test1),callbacks = [stop_early])


tuner.results_summary()

### CNN + att + LSTM + CNN

In [ ]:
LOG_DIR = f"{int(time.time())}"
seed_value= 0

def build_model(hp):

  reproduceResult()

  print('Ya it comes here')

  # SEARCH VALUES
  att_size = hp.Int("attention size",min_value =16, max_value = 128, step = 16)
  cnn_1_size = hp.Int("cnn1 size",min_value =16, max_value = 96, step = 16)
  cnn_1_dropout = hp.Float("cnn1 dropout",min_value = 0.1,max_value = 0.3,step = 0.1)

  lstm_size = hp.Int("lstm size",min_value =32, max_value = 256, step = 32)
  lstm_dropout = hp.Float("lstm dropout",min_value = 0.1,max_value = 0.5,step = 0.1)

  cnn_2_size = hp.Int("cnn2 size",min_value =32, max_value = 256, step = 32)
  cnn_2_dropout = hp.Float("cnn2 dropout",min_value = 0.1,max_value = 0.5,step = 0.1)

  lr = hp.Float("learning rate",min_value = 2e-5 ,max_value = 1e-3,step = 2e-5)


  # INPUT DEFINITION
  seq_input = keras.layers.Input(shape=(MAX_LEN,))

  # EMBEDDING
  embedded = keras.layers.Embedding(vocab_size,
                          embed_num_dims,
                          input_length = MAX_LEN,
                          weights = [embedd_matrix])(seq_input)


  # 1ST CNN LAYER
  cnn_1 = keras.layers.Conv1D(cnn_1_size,3)(embedded)
  cnn_1 = keras.layers.Activation(activation='relu')(cnn_1)
  cnn_1 = keras.layers.BatchNormalization()(cnn_1)
  cnn_1 = keras.layers.Dropout(cnn_1_dropout,seed=seed_value)(cnn_1)

  # ATTENTION
  attention = keras.layers.TimeDistributed(keras.layers.Dense(att_size))(cnn_1)
  attention = keras.layers.Reshape((128,att_size))(attention)
  attention = keras.layers.Activation('relu', name = 'cnn1_attention')(attention)
  attention_output = keras.layers.Dot(axes = 1)([cnn_1, attention])

  # LSTM LAYER
  lstm = keras.layers.LSTM(lstm_size,return_sequences=True,input_shape =(128,))(attention_output)
  lstm = keras.layers.Activation(activation='relu')(lstm)
  lstm = keras.layers.BatchNormalization()(lstm)
  lstm = keras.layers.Dropout(lstm_dropout,seed=seed_value)(lstm)

  # 2ND CNN LAYER
  cnn_2 = keras.layers.Conv1D(cnn_2_size,3)(lstm)
  cnn_2 = keras.layers.Activation(activation='relu')(cnn_2)
  cnn_2 = keras.layers.BatchNormalization()(cnn_2)
  cnn_2 = keras.layers.Dropout(cnn_2_dropout,seed=seed_value)(cnn_2)

  max_pooling = keras.layers.GlobalMaxPooling1D()(cnn_2)
  output = keras.layers.Dense(6, activation='softmax')(max_pooling)

  model = keras.Model(inputs = [seq_input], outputs = output)
  model.compile(loss='sparse_categorical_crossentropy', optimizer=keras.optimizers.Adam(learning_rate=lr), metrics=['accuracy'])

  return model

# EARLY STOPPING
stop_early = tf.keras.callbacks.EarlyStopping(monitor='val_accuracy',
                              patience=4,
                              restore_best_weights=True,
                              verbose=0, mode='max')


# PARAMETER SEARCH
tuner = RandomSearch(
    build_model,
    objective = keras_tuner.Objective('val_accuracy', direction="max"),
    max_trials = 10,
    executions_per_trial = 1,
    directory = LOG_DIR
    )

tuner.search(x=X_train1,y = y_train1,epochs = 20, batch_size = 8, validation_data = (X_test1,y_test1),callbacks = [stop_early])


tuner.results_summary()

### CNN + A + LSTM + CNN + CNN

In [ ]:
LOG_DIR = f"{int(time.time())}"
seed_value= 0

def build_model(hp):

  reproduceResult()

  print('Ya it comes here')

  # SEARCH VALUES
  att_size = hp.Int("attention size",min_value =16, max_value = 128, step = 16)
  cnn_1_size = hp.Int("cnn1 size",min_value =16, max_value = 96, step = 16)
  cnn_1_dropout = hp.Float("cnn1 dropout",min_value = 0.1,max_value = 0.3,step = 0.1)

  lstm_size = hp.Int("lstm size",min_value =32, max_value = 256, step = 32)
  lstm_dropout = hp.Float("lstm dropout",min_value = 0.1,max_value = 0.5,step = 0.1)
  cnn_2_size = hp.Int("cnn2 size",min_value =32, max_value = 256, step = 32)
  cnn_2_dropout = hp.Float("cnn2 dropout",min_value = 0.1,max_value = 0.5,step = 0.1)

  cnn_3_size = hp.Int("cnn3 size",min_value =32, max_value = 256, step = 32)
  cnn_3_dropout = hp.Float("cnn3 dropout",min_value = 0.1,max_value = 0.5,step = 0.1)

  lr = hp.Float("learning rate",min_value = 2e-5 ,max_value = 1e-3,step = 2e-5)


  # INPUT DEFINITION
  seq_input = keras.layers.Input(shape=(MAX_LEN,))

  # EMBEDDING
  embedded = keras.layers.Embedding(vocab_size,
                          embed_num_dims,
                          input_length = MAX_LEN,
                          weights = [embedd_matrix])(seq_input)

  # 1ST CNN LAYER
  cnn_1 = keras.layers.Conv1D(cnn_1_size,3)(embedded)
  cnn_1 = keras.layers.Activation(activation='relu')(cnn_1)
  cnn_1 = keras.layers.BatchNormalization()(cnn_1)
  cnn_1 = keras.layers.Dropout(cnn_1_dropout,seed=seed_value)(cnn_1)

  # ATTENTION
  attention = keras.layers.TimeDistributed(keras.layers.Dense(att_size))(cnn_1)
  attention = keras.layers.Reshape((128,att_size))(attention)
  attention = keras.layers.Activation('relu', name = 'cnn1_attention')(attention)
  attention_output = keras.layers.Dot(axes = 1)([cnn_1, attention])

  # LSTM LAYER
  lstm = keras.layers.LSTM(lstm_size, return_sequences=True,input_shape =(128,))(attention_output)
  lstm = keras.layers.Activation(activation='relu')(lstm)
  lstm = keras.layers.BatchNormalization()(lstm)
  lstm = keras.layers.Dropout(lstm_dropout,seed=seed_value)(lstm)

  # 2ND CNN LAYER
  cnn_2 = keras.layers.Conv1D(cnn_2_size,3)(lstm)
  cnn_2 = keras.layers.Activation(activation='relu')(cnn_2)
  cnn_2 = keras.layers.BatchNormalization()(cnn_2)
  cnn_2 = keras.layers.Dropout(cnn_2_dropout,seed=seed_value)(cnn_2)

  # 3ND CNN LAYER
  cnn_3 = keras.layers.Conv1D(cnn_3_size,3)(lstm)
  cnn_3 = keras.layers.Activation(activation='relu')(cnn_3)
  cnn_3 = keras.layers.BatchNormalization()(cnn_3)
  cnn_3 = keras.layers.Dropout(cnn_2_dropout,seed=seed_value)(cnn_3)

  max_pooling = keras.layers.GlobalMaxPooling1D()(cnn_3)
  output = keras.layers.Dense(6, activation='softmax')(max_pooling)

  model = keras.Model(inputs = [seq_input], outputs = output)
  model.compile(loss='sparse_categorical_crossentropy', optimizer=keras.optimizers.Adam(learning_rate=lr), metrics=['accuracy'])

  return model


# EARLY STOPPING
stop_early = tf.keras.callbacks.EarlyStopping(monitor='val_accuracy',
                              patience=4,
                              restore_best_weights=True,
                              verbose=0, mode='max')

# PARAMETER SEARCH
tuner = RandomSearch(
    build_model,
    objective = keras_tuner.Objective('val_accuracy', direction="max"),
    max_trials = 10,
    executions_per_trial = 1,
    directory = LOG_DIR
    )

tuner.search(x=X_train1,y = y_train1,epochs = 20, batch_size = 8, validation_data = (X_test1,y_test1),callbacks = [stop_early])


tuner.results_summary()

### CNN + att + LSTM + CNN + LSTM

In [ ]:
LOG_DIR = f"{int(time.time())}"
seed_value= 0

def build_model(hp):

  reproduceResult()

  print('Ya it comes here')

  # SEARCH VALUES
  att_size = hp.Int("attention size",min_value =16, max_value = 128, step = 16)
  cnn_1_size = hp.Int("cnn1 size",min_value =16, max_value = 96, step = 16)
  cnn_1_dropout = hp.Float("cnn1 dropout",min_value = 0.1,max_value = 0.3,step = 0.1)

  lstm_1_size = hp.Int("lstm1 size",min_value =32, max_value = 256, step = 32)
  lstm_1_dropout = hp.Float("lstm1 dropout",min_value = 0.1,max_value = 0.5,step = 0.1)

  cnn_2_size = hp.Int("cnn2 size",min_value =32, max_value = 256, step = 32)
  cnn_2_dropout = hp.Float("cnn2 dropout",min_value = 0.1,max_value = 0.5,step = 0.1)

  lstm_2_size = hp.Int("lstm2 size",min_value =32, max_value = 256, step = 32)
  lstm_2_dropout = hp.Float("lstm2 dropout",min_value = 0.1,max_value = 0.5,step = 0.1)

  lr = hp.Float("learning rate",min_value = 2e-5 ,max_value = 1e-3,step = 2e-5)


  # INPUT DEFINITION
  seq_input = keras.layers.Input(shape=(MAX_LEN,))

  # EMBEDDING
  embedded = keras.layers.Embedding(vocab_size,
                          embed_num_dims,
                          input_length = MAX_LEN,
                          weights = [embedd_matrix])(seq_input)


  # 1ST CNN LAYER
  cnn_1 = keras.layers.Conv1D(cnn_1_size,3)(embedded)
  cnn_1 = keras.layers.Activation(activation='relu')(cnn_1)
  cnn_1 = keras.layers.BatchNormalization()(cnn_1)
  cnn_1 = keras.layers.Dropout(cnn_1_dropout,seed=seed_value)(cnn_1)

  # ATTENTION
  attention = keras.layers.TimeDistributed(keras.layers.Dense(att_size))(cnn_1)
  attention = keras.layers.Reshape((128,att_size))(attention)
  attention = keras.layers.Activation('relu', name = 'cnn1_attention')(attention)
  attention_output = keras.layers.Dot(axes = 1)([cnn_1, attention])

  # LSTM LAYER
  lstm_1 = keras.layers.LSTM(lstm_1_size,return_sequences=True,input_shape =(128,))(attention_output)
  lstm_1 = keras.layers.Activation(activation='relu')(lstm_1)
  lstm_1 = keras.layers.BatchNormalization()(lstm_1)
  lstm_1 = keras.layers.Dropout(lstm_1_dropout,seed=seed_value)(lstm_1)

  # 2ND CNN LAYER
  cnn_2 = keras.layers.Conv1D(cnn_2_size,3)(lstm_1)
  cnn_2 = keras.layers.Activation(activation='relu')(cnn_2)
  cnn_2 = keras.layers.BatchNormalization()(cnn_2)
  cnn_2 = keras.layers.Dropout(cnn_2_dropout,seed=seed_value)(cnn_2)

  # 2ND LSTM LAYER
  lstm_2 = keras.layers.LSTM(lstm_1_size,return_sequences=True,input_shape =(128,))(attention_output)
  lstm_2 = keras.layers.Activation(activation='relu')(lstm_2)
  lstm_2 = keras.layers.BatchNormalization()(lstm_2)
  lstm_2 = keras.layers.Dropout(lstm_2_dropout,seed=seed_value)(lstm_2)

  max_pooling = keras.layers.GlobalMaxPooling1D()(lstm_2)
  output = keras.layers.Dense(6, activation='softmax')(max_pooling)

  model = keras.Model(inputs = [seq_input], outputs = output)
  model.compile(loss='sparse_categorical_crossentropy', optimizer=keras.optimizers.Adam(learning_rate=lr), metrics=['accuracy'])

  return model

# EARLY STOPPING
stop_early = tf.keras.callbacks.EarlyStopping(monitor='val_accuracy',
                              patience=4,
                              restore_best_weights=True,
                              verbose=0, mode='max')


# PARAMETER SEARCH
tuner = RandomSearch(
    build_model,
    objective = keras_tuner.Objective('val_accuracy', direction="max"),
    max_trials = 10,
    executions_per_trial = 1,
    directory = LOG_DIR
    )

tuner.search(x=X_train1,y = y_train1,epochs = 20, batch_size = 8, validation_data = (X_test1,y_test1),callbacks = [stop_early])


tuner.results_summary()

## Focal loss

Incorporating gamma into the hyperparameter search for the best-performing fusion architecture - CNN + A + LSTM + CNN + CNN, to evaluate whether it improves the performance of the model for the differently class distributed datasets.

### original train data

In [21]:
!pip install focal_loss
from focal_loss import SparseCategoricalFocalLoss

In [ ]:
LOG_DIR = f"{int(time.time())}"
seed_value= 0

def focal_loss_model(hp):

  reproduceResult()

  print('Ya it comes here')


  lr = hp.Float("learning rate",min_value = 2e-5 ,max_value = 1e-3,step = 2e-5)
  gamma = hp.Float("gamma",min_value = 0 ,max_value = 5,step = 0.5)


  # INPUT DEFINITION
  seq_input = keras.layers.Input(shape=(MAX_LEN,))

  # EMBEDDING
  embedded = keras.layers.Embedding(vocab_size,
                          embed_num_dims,
                          input_length = MAX_LEN,
                          weights = [embedd_matrix])(seq_input)

  # 1ST CNN LAYER
  cnn_1 = keras.layers.Conv1D(32,3)(embedded)
  cnn_1 = keras.layers.Activation(activation='relu')(cnn_1)
  cnn_1 = keras.layers.BatchNormalization()(cnn_1)
  cnn_1 = keras.layers.Dropout(0.1,seed=seed_value)(cnn_1)

  # ATTENTION
  attention = keras.layers.TimeDistributed(keras.layers.Dense(128))(cnn_1)
  attention = keras.layers.Reshape((128,128))(attention)
  attention = keras.layers.Activation('relu', name = 'cnn1_attention')(attention)
  attention_output = keras.layers.Dot(axes = 1)([cnn_1, attention])

  # LSTM LAYER
  lstm = keras.layers.LSTM(256, return_sequences=True,input_shape =(128,))(attention_output)
  lstm = keras.layers.Activation(activation='relu')(lstm)
  lstm = keras.layers.BatchNormalization()(lstm)
  lstm = keras.layers.Dropout(0.2,seed=seed_value)(lstm)

  # 2ND CNN LAYER
  cnn_2 = keras.layers.Conv1D(224,3)(lstm)
  cnn_2 = keras.layers.Activation(activation='relu')(cnn_2)
  cnn_2 = keras.layers.BatchNormalization()(cnn_2)
  cnn_2 = keras.layers.Dropout(0.5,seed=seed_value)(cnn_2)

  # 3ND CNN LAYER
  cnn_3 = keras.layers.Conv1D(224,3)(lstm)
  cnn_3 = keras.layers.Activation(activation='relu')(cnn_3)
  cnn_3 = keras.layers.BatchNormalization()(cnn_3)
  cnn_3 = keras.layers.Dropout(0.3,seed=seed_value)(cnn_3)

  max_pooling = keras.layers.GlobalMaxPooling1D()(cnn_3)
  output = keras.layers.Dense(6, activation='softmax')(max_pooling)

  model = keras.Model(inputs = [seq_input], outputs = output)
  model.compile(loss=SparseCategoricalFocalLoss(gamma = gamma), optimizer=keras.optimizers.Adam(learning_rate=lr), metrics=['accuracy'])

  return model


# EARLY STOPPING
stop_early = tf.keras.callbacks.EarlyStopping(monitor='val_accuracy',
                              patience=4,
                              restore_best_weights=True,
                              verbose=0, mode='max')

# PARAMETER SEARCH
tuner = RandomSearch(
    focal_loss_model,
    objective = keras_tuner.Objective('val_accuracy', direction="max"),
    max_trials = 10,
    executions_per_trial = 1,
    directory = LOG_DIR
    )

tuner.search(x=X_train1,y = y_train1,epochs = 20, batch_size = 8, validation_data = (X_test1,y_test1),callbacks = [stop_early])


tuner.results_summary()

### filtered 150

In [ ]:
filtered_150['lemmatized text'] = filtered_150.text.apply(lambda x: lemmatization(x))

# MAX_FEATURES = 6000
embed_num_dims = 300
tokenizer = Tokenizer()
tokenizer.fit_on_texts(filtered_150['lemmatized text'])
index_of_words = tokenizer.word_index
vocab_size = len(index_of_words) + 1

tokenized_train = tokenizer.texts_to_sequences(filtered_150['lemmatized text'])

RNN_CELL_SIZE = 32

MAX_LEN = 130
X_train1 = pad_sequences(tokenized_train, maxlen=MAX_LEN,padding='pre')
y_train1 = filtered_150['category']


embedd_matrix = create_embedding_matrix(index_of_words, embed_num_dims)

LOG_DIR = f"{int(time.time())}"
seed_value= 0

# PARAMETER SEARCH
tuner = RandomSearch(
    focal_loss_model,
    objective = keras_tuner.Objective('val_accuracy', direction="max"),
    max_trials = 10,
    executions_per_trial = 1,
    directory = LOG_DIR
    )

tuner.search(x=X_train1,y = y_train1,epochs = 20, batch_size = 8, validation_data = (X_test1,y_test1),callbacks = [stop_early])


tuner.results_summary()

### filtered & augmented 150

In [ ]:
aug_train150['lemmatized text'] = aug_train150.text.apply(lambda x: lemmatization(x))

# MAX_FEATURES = 6000
embed_num_dims = 300
tokenizer = Tokenizer()
tokenizer.fit_on_texts(aug_train150['lemmatized text'])
index_of_words = tokenizer.word_index
vocab_size = len(index_of_words) + 1

tokenized_train = tokenizer.texts_to_sequences(aug_train150['lemmatized text'])

RNN_CELL_SIZE = 32

MAX_LEN = 130
X_train1 = pad_sequences(tokenized_train, maxlen=MAX_LEN,padding='pre')
y_train1 = aug_train150['category']


embedd_matrix = create_embedding_matrix(index_of_words, embed_num_dims)

LOG_DIR = f"{int(time.time())}"
seed_value= 0

# PARAMETER SEARCH
tuner = RandomSearch(
    focal_loss_model,
    objective = keras_tuner.Objective('val_accuracy', direction="max"),
    max_trials = 10,
    executions_per_trial = 1,
    directory = LOG_DIR
    )

tuner.search(x=X_train1,y = y_train1,epochs = 20, batch_size = 8, validation_data = (X_test1,y_test1),callbacks = [stop_early])


tuner.results_summary()

### filtered 312

In [ ]:
filtered_312['lemmatized text'] = filtered_150.text.apply(lambda x: lemmatization(x))

# MAX_FEATURES = 6000
embed_num_dims = 300
tokenizer = Tokenizer()
tokenizer.fit_on_texts(filtered_312['lemmatized text'])
index_of_words = tokenizer.word_index
vocab_size = len(index_of_words) + 1

tokenized_train = tokenizer.texts_to_sequences(filtered_312['lemmatized text'])

RNN_CELL_SIZE = 32

MAX_LEN = 130
X_train1 = pad_sequences(tokenized_train, maxlen=MAX_LEN,padding='pre')
y_train1 = filtered_312['category']


embedd_matrix = create_embedding_matrix(index_of_words, embed_num_dims)

LOG_DIR = f"{int(time.time())}"
seed_value= 0

# PARAMETER SEARCH
tuner = RandomSearch(
    focal_loss_model,
    objective = keras_tuner.Objective('val_accuracy', direction="max"),
    max_trials = 10,
    executions_per_trial = 1,
    directory = LOG_DIR
    )

tuner.search(x=X_train1,y = y_train1,epochs = 20, batch_size = 8, validation_data = (X_test1,y_test1),callbacks = [stop_early])


tuner.results_summary()

### filtered & augmented 312

In [ ]:
aug_train312['lemmatized text'] = aug_train312.text.apply(lambda x: lemmatization(x))

# MAX_FEATURES = 6000
embed_num_dims = 300
tokenizer = Tokenizer()
tokenizer.fit_on_texts(aug_train312['lemmatized text'])
index_of_words = tokenizer.word_index
vocab_size = len(index_of_words) + 1

tokenized_train = tokenizer.texts_to_sequences(aug_train312['lemmatized text'])

RNN_CELL_SIZE = 32

MAX_LEN = 130
X_train1 = pad_sequences(tokenized_train, maxlen=MAX_LEN,padding='pre')
y_train1 = aug_train312['category']


embedd_matrix = create_embedding_matrix(index_of_words, embed_num_dims)

LOG_DIR = f"{int(time.time())}"
seed_value= 0

# PARAMETER SEARCH
tuner = RandomSearch(
    focal_loss_model,
    objective = keras_tuner.Objective('val_accuracy', direction="max"),
    max_trials = 10,
    executions_per_trial = 1,
    directory = LOG_DIR
    )

tuner.search(x=X_train1,y = y_train1,epochs = 20, batch_size = 8, validation_data = (X_test1,y_test1),callbacks = [stop_early])


tuner.results_summary()

# Final evaluation - Classification report

In [57]:
embed_num_dims = 300
tokenizer = Tokenizer()
tokenizer.fit_on_texts(train_data['lemmatized text'])
index_of_words = tokenizer.word_index
vocab_size = len(index_of_words) + 1

tokenized_train = tokenizer.texts_to_sequences(train_data['lemmatized text'])
tokenized_test = tokenizer.texts_to_sequences(test_data['lemmatized text'])


RNN_CELL_SIZE = 32

MAX_LEN = 130
X_train1 = pad_sequences(tokenized_train, maxlen=MAX_LEN,padding='pre')
y_train1 = train_data['category']

X_test1 = pad_sequences(tokenized_test, maxlen=MAX_LEN,padding='pre')
y_test1 = test_data['category']

embedd_matrix = create_embedding_matrix(index_of_words, embed_num_dims)

def final_fusion_model():

  reproduceResult()

  # INPUT DEFINITION
  seq_input = keras.layers.Input(shape=(MAX_LEN,))

  # EMBEDDING
  embedded = keras.layers.Embedding(vocab_size,
                          embed_num_dims,
                          input_length = MAX_LEN,
                          weights = [embedd_matrix])(seq_input)

  # 1ST CNN LAYER
  cnn_1 = keras.layers.Conv1D(32,3)(embedded)
  cnn_1 = keras.layers.Activation(activation='relu')(cnn_1)
  cnn_1 = keras.layers.BatchNormalization()(cnn_1)
  cnn_1 = keras.layers.Dropout(0.1,seed=seed_value)(cnn_1)

  # ATTENTION
  attention = keras.layers.TimeDistributed(keras.layers.Dense(128))(cnn_1)
  attention = keras.layers.Reshape((128,128))(attention)
  attention = keras.layers.Activation('relu', name = 'cnn1_attention')(attention)
  attention_output = keras.layers.Dot(axes = 1)([cnn_1, attention])

  # LSTM LAYER
  lstm = keras.layers.LSTM(256, return_sequences=True,input_shape =(128,))(attention_output)
  lstm = keras.layers.Activation(activation='relu')(lstm)
  lstm = keras.layers.BatchNormalization()(lstm)
  lstm = keras.layers.Dropout(0.2,seed=seed_value)(lstm)

  # 2ND CNN LAYER
  cnn_2 = keras.layers.Conv1D(224,3)(lstm)
  cnn_2 = keras.layers.Activation(activation='relu')(cnn_2)
  cnn_2 = keras.layers.BatchNormalization()(cnn_2)
  cnn_2 = keras.layers.Dropout(0.5,seed=seed_value)(cnn_2)

  # 3ND CNN LAYER
  cnn_3 = keras.layers.Conv1D(224,3)(lstm)
  cnn_3 = keras.layers.Activation(activation='relu')(cnn_3)
  cnn_3 = keras.layers.BatchNormalization()(cnn_3)
  cnn_3 = keras.layers.Dropout(0.3,seed=seed_value)(cnn_3)

  max_pooling = keras.layers.GlobalMaxPooling1D()(cnn_3)
  output = keras.layers.Dense(6, activation='softmax')(max_pooling)

  model = keras.Model(inputs = [seq_input], outputs = output)
  model.compile(loss=SparseCategoricalFocalLoss(gamma = 3), optimizer=keras.optimizers.Adam(learning_rate=0.00024), metrics=['accuracy'])

  return model


In [ ]:
final_fusion_model = final_fusion_model()

from keras.callbacks import ModelCheckpoint

# Define the filepath to save the model with the HIGHEST accuracy
filepath = 'best_model.h5'

# Create the ModelCheckpoint callback
checkpoint = ModelCheckpoint(filepath, monitor='val_accuracy', save_best_only=True, mode='max')

final_fusion_model.fit(X_train1, y_train1, epochs=50, batch_size=8, validation_data=(X_test1, y_test1), callbacks=[checkpoint])

In [ ]:
from keras.models import load_model

# Load the model from the saved filepath
final_fusionmodel = load_model('best_model.h5')

# Get total predictions for the test data
predictions = final_fusionmodel.predict(X_test1)

predicted_labels = predictions.argmax(axis=1)

In [ ]:
predicted_labels

In [ ]:
from sklearn.metrics import recall_score
from sklearn.metrics import classification_report

true_labels = np.array(y_test1, dtype = int)

confusion = confusion_matrix(true_labels, predicted_labels)
print("\nConfusion Matrix:\n", confusion)

label_names = ['Obligation', 'Omission','Permission', 'Power', 'Prohibition', 'Right']
print(classification_report(true_labels, predicted_labels, target_names=label_names))